In [1]:
%load_ext autoreload
%autoreload 2

import os, sys

sys.path.insert(0, "src")

from pandas import DataFrame
from analytics.metrics.metric import MetricCalculator
from analytics.metrics.time import TimeMetrics
from spec.enums import MainTableColumns as Cols
from spec.spec_definition import PS2Versions

In [2]:
from database.config import PS2DataConfig

spec = PS2Versions.v1_0.load()

data_config = PS2DataConfig.from_yaml("sample_data_configs/cwo_s19.yaml", spec)


In [3]:
from analytics.ps2_dataset import PS2Dataset

dataset = PS2Dataset(spec, data_config)

In [ ]:
# TODO: Add timezone preprocessor to the dataset

main_table = dataset.get_main_table()
main_table.head()

,Order,SubjectID,ToolInstances,ServerTimestamp,ServerTimezone,CourseID,CourseSectionID,AssignmentID,ProblemID,Attempt,CodeStateID,IsEventOrderingConsistent,EventType,Score,Compile.Result,CompileMessageType,CompileMessageData,EventID,ParentEventID,SourceLocation
0,0,f40a635828e2bffd0a598a7ed621fc93,Java 8; CodeWorkout,2019-03-20 19:34:25,0,CS 1,1,492.0,32,25,548938774dc948448b0bb51139bbcae3a3966ab8,True,Run.Program,1.0,NaN,NaN,NaN,32-84779,NaN,NaN
1,1,f40a635828e2bffd0a598a7ed621fc93,Java 8; CodeWorkout,2019-03-20 19:34:25,0,CS 1,1,492.0,32,25,548938774dc948448b0bb51139bbcae3a3966ab8,True,Compile,NaN,Success,NaN,NaN,32-84780,32-84779,NaN
2,2,f18374bf891c9af64c8570448824202d,Java 8; CodeWorkout,2019-03-20 21:02:00,0,CS 1,2,492.0,32,1,20dc599067ac07408b5636973fc6b8a37db9af32,True,Run.Program,1.0,NaN,NaN,NaN,32-84781,NaN,NaN
3,3,f18374bf891c9af64c8570448824202d,Java 8; CodeWorkout,2019-03-20 21:02:00,0,CS 1,2,492.0,32,1,20dc599067ac07408b5636973fc6b8a37db9af32,True,Compile,NaN,Success,NaN,NaN,32-84782,32-84781,NaN
4,4,4d230b683bf9840553ae57f4acc96e81,Java 8; CodeWorkout,2019-03-20 21:13:09,0,CS 1,1,492.0,32,1,23c57b36002c4d482df3d25f97778f74a2ed581a,True,Run.Program,0.0,NaN,NaN,NaN,32-84783,NaN,NaN


In [6]:
main_table.ServerTimestamp.iloc[0]

Timestamp('2019-03-20 19:34:25')

In [7]:
student_subset = main_table[Cols.SubjectID].unique()[:50]
mt_subset = main_table[main_table[Cols.SubjectID].isin(student_subset)]

In [38]:
time_metrics = TimeMetrics(5*60, 30 * 60, True, Cols.ServerTimestamp)

# time_metrics.test_calculation(mt_subset, [Cols.SubjectID, Cols.ProblemID])

result = mt_subset.groupby([Cols.SubjectID, Cols.ProblemID]).apply(lambda x: time_metrics.calculate(x))
result

C:\Users\twprice\AppData\Local\Temp\ipykernel_18296\1871496015.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = mt_subset.groupby([Cols.SubjectID, Cols.ProblemID]).apply(lambda x: time_metrics.calculate(x))


ActiveTime  IdleTime  TotalTime  \
SubjectID                        ProblemID                                    
04c32d4d95425f73b3a1d6502aed4d48 1                 0.0       0.0        0.0   
                                 3                73.0       0.0       73.0   
                                 5                56.0       0.0       56.0   
                                 12                0.0       0.0        0.0   
                                 13               83.0       0.0       83.0   
...                                                ...       ...        ...   
fc5f86251458722c799d1830fa0c2c1f 232             259.0       0.0      259.0   
                                 233             147.0       0.0      147.0   
                                 234              12.0       0.0       12.0   
                                 235               0.0       0.0        0.0   
                                 236             570.0     366.0      936.0   

                                            ActiveTimeAfterCorrect  #Breaks  \
SubjectID                        ProblemID                                    
04c32d4d95425f73b3a1d6502aed4d48 1                             0.0        0   
                                 3                             0.0        0   
                                 5                             0.0        0   
                                 12                            0.0        0   
                                 13                            0.0        0   
...                                                            ...      ...   
fc5f86251458722c799d1830fa0c2c1f 232                           0.0        0   
                                 233                           0.0        0   
                                 234                           0.0        0   
                                 235                           0.0        0   
                                 236                           0.0        0   

                                                     StartTime  \
SubjectID                        ProblemID                       
04c32d4d95425f73b3a1d6502aed4d48 1         2019-02-23 22:44:51   
                                 3         2019-02-23 22:49:34   
                                 5         2019-02-23 22:18:45   
                                 12        2019-02-23 22:54:58   
                                 13        2019-02-23 21:19:38   
...                                                        ...   
fc5f86251458722c799d1830fa0c2c1f 232       2019-02-24 15:05:33   
                                 233       2019-02-21 20:34:04   
                                 234       2019-02-21 19:48:14   
                                 235       2019-02-21 19:53:49   
                                 236       2019-02-21 20:01:41   

                                              FirstCorrectTime  \
SubjectID                        ProblemID                       
04c32d4d95425f73b3a1d6502aed4d48 1         2019-02-23 22:44:51   
                                 3         2019-02-23 22:50:47   
                                 5         2019-02-23 22:19:41   
                                 12        2019-02-23 22:54:58   
                                 13        2019-02-23 21:21:01   
...                                                        ...   
fc5f86251458722c799d1830fa0c2c1f 232       2019-02-24 15:09:52   
                                 233       2019-02-21 20:36:31   
                                 234       2019-02-21 19:48:26   
                                 235       2019-02-21 19:53:49   
                                 236       2019-02-21 20:17:17   

                                                       EndTime   n  
SubjectID                        ProblemID                          
04c32d4d95425f73b3a1d6502aed4d48 1         2019-02-23 22:44:51   2  
                                 3         2019-02-23 22:50:47 

In [39]:
result.describe()

,ActiveTime,IdleTime,TotalTime,ActiveTimeAfterCorrect,#Breaks,StartTime,FirstCorrectTime,EndTime,n
count,2364.000000,2364.000000,2364.000000,2364.000000,2364.000000,2364,2290,2364,2364.000000
mean,138.511421,100.289340,238.800761,4.704315,0.099408,2019-03-24 23:30:15.665820672,2019-03-24 21:29:29.770742528,2019-03-25 00:46:49.351945984,11.879865
min,0.000000,0.000000,0.000000,0.000000,0.000000,2019-02-20 21:13:37,2019-02-20 21:17:01,2019-02-20 21:17:01,2.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,2019-03-05 00:17:14.249999872,2019-03-05 00:18:13.750000128,2019-03-05 00:52:41.500000,2.000000
50%,22.000000,0.000000,25.000000,0.000000,0.000000,2019-03-21 18:51:30.500000,2019-03-21 18:33:32.500000,2019-03-21 19:55:27,5.000000
75%,152.250000,0.000000,208.000000,0.000000,0.000000,2019-04-10 19:18:17.249999872,2019-04-10 19:43:00.500000,2019-04-10 19:41:09.500000,13.000000
max,2823.000000,3898.000000,5033.000000,604.000000,7.000000,2019-05-06 03:39:05,2019-05-06 03:39:05,2019-05-06 03:39:05,556.000000
std,276.861655,357.207892,532.070661,33.272457,0.375757,NaN,NaN,NaN,22.669084
